# # MST

### # Graph and Other Data Structures

- load skt - DONE
- load dcs - DONE


- create the full graph - sktwseg_utf8 - DONE
- Nodelist - DONE
- Features-Adjacency Matrix - DONE
- Weight-Adjacency Matrix - DONE
- Conflict Matrix - or - dictionary (since it's sparse) - DONE


- new paths base on cng probability - DONE


### TODO: 

- Make a table of MST scores of various files throughout time/iterations
- Check the gradient descent + backpropagation formulation
- Take full size vector

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
## bUILT-iN pACKAGES
import pickle
from collections import defaultdict
import json
import numpy as np
import math

## lAST sUMMER
from romtoslp import *
from sentences import *
from DCS import *
import MatDB

## lAST yEAR
# from word_definite import *
# from nnet import *
# from heap_n_PrimMST import *
# from word_definite import *

In [3]:
def organize_mst_nodes(mst_nodes):
    sol_lemmas = []
    sol_cngs = []
    print()
    total_tree_weight = 0
    for key, val in mst_nodes.items():
        print('{', end = '')
        for node in val:
            print(node.lemma, (node.cng, node.derived, node.src,), end = ', ')
            if node.src != -1:
                total_tree_weight += WScalarMat[node.src][node.id]
        print('};', end = ' ')
    print()
    print('Weight of the spanning tree: ', total_tree_weight)

In [5]:
import Train_n_Save_NNet
# Train_n_Save_NNet.trainer.Reset()
# Train_n_Save_NNet.trainer.Load('outputs/saved_trainer.p')


In [6]:
loaded_SKT = pickle.load(open('../Simultaneous_CompatSKT_10K.p', 'rb'))
loaded_DCS = pickle.load(open('../Simultaneous_DCS_10K.p', 'rb'))

In [7]:
matDB = MatDB.MatDB()

In [68]:
Train_n_Save_NNet.InitModule(matDB)
Train_n_Save_NNet.trainer

443
0.0378189652936


In [69]:
Train_n_Save_NNet.main(loaded_SKT, loaded_DCS)

ITERATION OUT 0
Permutation:  [8291 6852 9107 6385 8265 1935 5085  529 2264 6705 2675 5082 9096 6143 2160
 7590  582  840 7651 9484 1885 1857 8071 3271 3228 9414  876 4034 4909 3877]
ITERATION IN 0

FileKey: 384497, Loss: -0.933, Original MSTScore:  6.000
list index out of range
list index out of range

FileKey: 121245, Loss: -1.156, Original MSTScore:  6.000

FileKey: 27945, Loss: -3.305, Original MSTScore:  5.304

FileKey: 370782, Loss: -2.218, Original MSTScore:  4.862

FileKey: 94110, Loss: -7.506, Original MSTScore: 15.848

FileKey: 355352, Loss:   -inf, Original MSTScore:  0.000

FileKey: 197525, Loss: 71.531, Original MSTScore: 1940.736

FileKey: 6833, Loss: 203.014, Original MSTScore: 1231.969

FileKey: 46128, Loss:    inf, Original MSTScore:    inf

FileKey: 8646, Loss: -614.334, Original MSTScore: 1651.308
list index out of range

FileKey: 275495, Loss: 1095.496, Original MSTScore: 1774.153

FileKey: 28746, Loss: -74.802, Original MSTScore: 185.798

FileKey: 433898, Loss: 114

/home/bishal/Documents/Sanskrit_project/Bishal/Clean_n_clear/nnet.py:10: RuntimeWarning: overflow encountered in exp
  evec = 1 + np.exp(-vec)



FileKey: 123332, Loss: 1861.077, Original MSTScore: 3732.766
list index out of range

FileKey: 272287, Loss: 286.777, Original MSTScore: 461.026

FileKey: 277440, Loss: 289.504, Original MSTScore: 469.684

FileKey: 250016, Loss: 939.414, Original MSTScore: 2043.127

FileKey: 412263, Loss: 2012.696, Original MSTScore: 3149.821

FileKey: 24995, Loss: -520.841, Original MSTScore: 554.668

FileKey: 253157, Loss: 1721.761, Original MSTScore: 2612.676

FileKey: 32414, Loss: 1783.547, Original MSTScore: 3403.316

FileKey: 333238, Loss: -809.791, Original MSTScore: 2293.292

FileKey: 38591, Loss: -419.576, Original MSTScore: 1677.204

FileKey: 23222, Loss:    inf, Original MSTScore:    inf
ITERATION IN 1

FileKey: 384497, Loss:   -inf, Original MSTScore: 2552.672
list index out of range
list index out of range


/home/bishal/Documents/Sanskrit_project/Bishal/Clean_n_clear/Train_n_Save_NNet.py:165: RuntimeWarning: invalid value encountered in double_scalars
  Total_Loss += (W_star - GetMSTWeight(mst_nodes, WScalarMat))



FileKey: 121245, Loss:    nan, Original MSTScore:    inf

FileKey: 27945, Loss: 408.335, Original MSTScore: 2221.743

FileKey: 370782, Loss: -122.593, Original MSTScore: 2522.780

FileKey: 94110, Loss: 361.216, Original MSTScore: 2460.808

FileKey: 355352, Loss:   -inf, Original MSTScore:  0.000

FileKey: 197525, Loss:    nan, Original MSTScore:    inf

FileKey: 6833, Loss:    nan, Original MSTScore:    inf

FileKey: 46128, Loss:    inf, Original MSTScore:    inf

FileKey: 8646, Loss:    nan, Original MSTScore:    inf
list index out of range

FileKey: 275495, Loss:    inf, Original MSTScore:    inf

FileKey: 28746, Loss: 218.888, Original MSTScore: 464.282

FileKey: 433898, Loss:    nan, Original MSTScore:    inf

FileKey: 37601, Loss:   -inf, Original MSTScore: 928.509

FileKey: 432223, Loss:    inf, Original MSTScore:    inf

FileKey: 123332, Loss:    nan, Original MSTScore:    inf
list index out of range

FileKey: 272287, Loss: 515.933, Original MSTScore: 845.070

FileKey: 277440, 

KeyboardInterrupt: 

In [54]:
def printLikeTable(hist_d):
    print('{:^65}'.format("Training History"))
    print('='*65)
    for pfile, lossList in hist_d.items():
        print('{:>6}.p |'.format(pfile), end='') # 8
        for aloss in lossList:
            print('{:>10.3f}|'.format(aloss), end='') # iter(=5)*11 = 55
        print()
# printLikeTable(Train_n_Save_NNet.trainer.history)

In [55]:
# Train_n_Save_NNet.trainer.Load('outputs/saved_trainer.p')
printLikeTable(Train_n_Save_NNet.trainer.history)

                        Training History                         
151030.p |    -0.000|
365964.p |    -0.000|
387842.p |    -0.151|


In [ ]:
# Train_n_Save_NNet.trainer._full_cnglist

In [18]:
fk = '197583.p2'
dcsObj = loaded_DCS[fk]
sentenceObj = loaded_SKT[fk]

In [19]:
# display(mst_nodes)
SeeDCS(dcsObj)

DCS ANALYZE
---------------
zawkoRaM laGu tIkzRaM ca bfhadazwadalaM ca yat   
[['ṣaṣ', 'koṇa'], ['laghu'], ['tīkṣṇa'], ['ca'], ['bṛhant', 'aṣṭan', 'dala'], ['ca'], ['yad']]
Lemmas: ['zaz', 'koRa', 'laGu', 'tIkzRa', 'ca', 'bfhant', 'azwan', 'dala', 'ca', 'yad']
[['3', '31'], ['31'], ['31'], ['2'], ['3', '3', '31'], ['2'], ['31']]



In [20]:
# Use 37171.p
print(sentenceObj.sent_id)
SeeSentence(sentenceObj)


197583
SKT ANALYZE
---------------
zawkoRaM laGu tIkzRaM ca bfhadazwadalaM ca yat   
Analyzing  zawkoRam
0 :  zaw ['zaz'] [{'compound': ['iic.']}]
0 :  zaw ['zaz'] [{'noun': ['acc. pl. *', 'nom. pl. *']}]
3 :  koRam ['koRa'] [{'noun': ['acc. sg. m.']}]
3 :  koRam ['koRa'] [{'noun': ['acc. sg. n.', 'nom. sg. n.']}]
Analyzing  laGu
0 :  laGu ['laGu'] [{'undetermined': ['adv.']}]
0 :  laGu ['laGu'] [{'noun': ['acc. sg. n.', 'nom. sg. n.']}]
0 :  laGu ['laGu'] [{'noun': ['voc. sg. n.']}]
Analyzing  tIkzRam
0 :  tIkzRam ['tIkzRa'] [{'noun': ['acc. sg. m.', 'acc. sg. n.', 'nom. sg. n.']}]
Analyzing  ca
0 :  ca ['ca'] [{'indeclinable': ['conj.']}]
0 :  ca ['ca'] [{'indeclinable': ['conj.']}]
Analyzing  bfhadazwadalam
0 :  bfhat ['bfhat', 'bfh', 'bfh', 'bfh', 'bfh'] [{'noun': ['acc. sg. n.', 'nom. sg. n.']}, {'verb': ['ppr. [6] ac.']}, {'noun': ['acc. sg. n.', 'nom. sg. n.']}, {'verb': ['ca. ppr. ac.']}, {'noun': ['acc. sg. n.', 'nom. sg. n.']}]
0 :  bfhat ['bfhat', 'bfh', 'bfh', 'bfh', 'bfh']

In [59]:
# Train_n_Save_NNet.trainer.neuralnet.U